<a href="https://colab.research.google.com/github/salsan1128/Predicting-Product-Ratings-from-Written-Reviews/blob/main/Predicting_Product_Ratings_Python_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wordcloud==1.8.1

import pandas as pd
import gzip
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import json

from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordcloud: filename=wordcloud-1.8.1-cp310-cp310-linux_x86_64.whl size=402046 sha256=24d4d5f9f8679a209fa543b6862b87997ef06fadbf1662707b3c7572a3d6ca05
  Stored in directory: /root/.cache/pip/wheels/6b/b1/75/024072f28a2b9d6c4f80b35ebfd6ced7fd9fc58e9f1b933b9f
Successfully built wordcloud
  Attempting uninstall: wordcloud
    Found existing installation: wordcloud 1.9.3
    Uninstalling wordcloud-1.9.3:
      Successfully uninstalled wordcloud-1.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


ImportError: ignored

In [ ]:
def parse(path):
 g = gzip.open(path, 'rb')
 for l in g:
   yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
    if i == 50000:
        break
  return pd.DataFrame.from_dict(df, orient='index')
df = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

FileNotFoundError: ignored

In [ ]:
test_df = df[['asin', 'reviewText', 'overall', 'summary', 'reviewTime']][:50000]
test_df1 = test_df.copy()
test_df1["reviewText"] = test_df["summary"] + ' ' + test_df["reviewText"]
test_df1 = test_df1.drop("summary", 1)


In [ ]:
test_df1["overall"] = test_df1["overall"].astype(str)
test_df1["overall"] = test_df1["overall"].replace({"5.0": "Good", "4.0": "Good", "3.0": "Neutral", "2.0": "Bad", "1.0": "Bad"})
test_df1["reviewText"] = test_df1["reviewText"].str.lower()
test_df1


In [ ]:
# First choice for how to deal with contractions

# def contractions(s):
#  s = re.sub(r"won't", "will not",s)
#  s = re.sub(r'wouldn\'t', 'would not',s)
#  s = re.sub(r'couldn\'t', 'could not',s)
#  s = re.sub(r'\'d', 'would',s)
#  s = re.sub(r'can\'t', 'can not',s)
#  s = re.sub(r'ain\'t', ' am not', s)
#  s = re.sub(r'n\'t', ' not', s)
#  s = re.sub(r'\'re', ' are', s)
#  s = re.sub(r'\'s', ' is', s)
#  s = re.sub(r'\'ll', ' will', s)
#  s = re.sub(r'\'t', ' not', s)
#  s = re.sub(r'\'ve', ' have', s)
#  s = re.sub(r'\'m', ' am', s)
#  return s


# Handling punctuation, special characters, and contractions

def special(s):
 s = re.sub('\\\\', '', s)
 s = re.sub("\\'", '', s)
 s = re.sub('[^A-Za-z0-9]+', ' ', s)
 return s


test_df1["reviewText"] = test_df1["reviewText"].apply(lambda x: special(x))

In [ ]:
# Getting the distribution of reviews by sentiment

a = test_df1['overall'].value_counts().plot.bar()
a.set_title("Distribution of User Reviews")

In [ ]:
# Getting the word cloud for all reviews

text = " ".join(review for review in test_df1['reviewText'])

stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords.update(["is, not, a, an, i, am, but, will, still, use"])


wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

plt.figure(figsize = (10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Getting the word cloud for good reviews

good_text = " ".join(review for review in test_df1[test_df1['overall'] == "Good"]['reviewText'])

wordcloud = WordCloud(max_words=100, stopwords=stopwords, background_color="white").generate(good_text)

plt.figure(figsize = (10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#Word cloud for bad reviews
bad_text = " ".join(review for review in test_df1[test_df1['overall'] == "Bad"]['reviewText'])

wordcloud = WordCloud(max_words=100, stopwords=stopwords, background_color="white").generate(bad_text)

plt.figure(figsize = (10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#Word clouds for neutral reviews
neutral_text = " ".join(review for review in test_df1[test_df1['overall'] == "Neutral"]['reviewText'])

wordcloud = WordCloud(max_words=100, stopwords=stopwords, background_color="white").generate(bad_text)

plt.figure(figsize = (10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Getting the top 10 products with the most reviews

test_df1 = test_df1.replace('B000T9VK56','Unisex T shirt')
test_df1 = test_df1.replace({'B0001ZNZJM': 'ExOfficio Men\'s Give-N-Go Boxer Brief Single Pack'})
test_df1 = test_df1.replace({'B00012O12A': 'Levi\'s Men\'s 505 Regular Fit Jeans'})
test_df1 = test_df1.replace({'B0008EOEPK': 'Lee Men\'s Regular Fit Straight Leg Jeans'})
test_df1 = test_df1.replace({'B0007YR8WW': 'Playtex Womens 18 Hour Original Comfort Strap Bra'})
test_df1 = test_df1.replace({'B0000C321X': 'Unisex Item'})
test_df1 = test_df1.replace({'B000KEG4V0 ':'Unisex Item'})
test_df1 = test_df1.replace({'B0007CKMA4': 'Wrangler Men Rugged Wear Relaxed Fit Jean'})
test_df1_Bar_MostReviews = test_df1.groupby('asin').count()["overall"].sort_values(ascending=False).head(10)

bar_MostReviews = test_df1_Bar_MostReviews.plot.barh()
bar_MostReviews.set_title("Top 10 Most Reviewed Products")
bar_MostReviews.set_ylabel("")
bar_MostReviews.set_ylabel("# of Reviews (in 1000s)")
bar_MostReviews.invert_yaxis()
bar_MostReviews


In [ ]:
%matplotlib inline
import pandas as pd

# Getting the top 10 highest rated products


test_df1 = test_df1.replace('B00011VJ7Q', 'Wigwam Hiking Outdoor F6077 Sock')
test_df1 = test_df1.replace({'B000H92UKM': 'Allen Edmonds Men\'s Combination Cedar Shoe Tree'})
test_df1 = test_df1.replace({'B000B6RWJ0': 'Sterling Silver Box Chain Necklace 0.8mm Very Thin Nickel Free Italy, Sizes 7-30 inch'})
test_df1 = test_df1.replace({'B00030B1GO': 'Kangol Men, Women Tropic 504 Ventair'})
test_df1 = test_df1.replace({'B0007NI1QK': 'Cluster Style Swarovski Elements Crystal Handmade Sterling Silver Leverback Drop Earrings'})
test_df1 = test_df1.replace({'B0009MZW34': 'thorlos womens Lthw Max Cushion Hiking Crew Socks'})
test_df1 = test_df1.replace({'B000NKC7O4': 'Unisex T shirt'})

#2. Create a cross tab for  the DF
test_df1_counting = pd.crosstab(test_df1['asin'], test_df1['overall'])
test_df2_c = test_df1_counting.assign(f = test_df1_counting['Good'] / (test_df1_counting['Bad'] + test_df1_counting['Good'] + test_df1_counting['Neutral'])).sort_values(['f', 'Good'], ascending=False).drop('f', axis = 1)


test_df2_Bar_HighestRated = test_df2_c[0:10]
bar_highest_rated = test_df2_Bar_HighestRated.plot.barh(legend=False)
bar_highest_rated.set_title("Top 10 Highest Rated Products")
bar_highest_rated.set_xlabel("# of Good Reviews (by 1000s)")
bar_highest_rated.set_ylabel("")
bar_highest_rated.invert_yaxis()

In [ ]:
#Stemming the reviews
ps = PorterStemmer()

def stemming(review):
     review = review.split()
     review = [ps.stem(word) for word in review]
     review = ' '.join(review)
     return review

test_df1["reviewText"] = test_df1["reviewText"].apply(lambda x: stemming(x))

In [ ]:
train, test = train_test_split(test_df1, test_size=0.2)

In [ ]:
# Running TF-IDF on our dataset

v = TfidfVectorizer(max_features=5000)
train_x = v.fit_transform(train['reviewText'])


df_vectorized = v.fit_transform(test_df1['reviewText'])
y = test_df1['overall']

In [ ]:
# Testing set (transform)
test_x = v.transform(test['reviewText'])


In [ ]:
# Running 5-fold cross validation and getting the best k
max_accuracy = 0
best_k = -1
for i in range(1, test_df1.size):
    neigh = KNeighborsClassifier(n_neighbors=i, weights='distance')
    neigh.fit(train_x, train['overall'])
    if (cross_val_score(neigh, test_x, test['overall']).mean() > max_accuracy):
        max_accuracy = cross_val_score(neigh, test_x, test['overall']).mean()
        best_k = i

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=best_k, weights='distance')
neigh.fit(train_x, train['overall'])
predicted = neigh.predict(test_x)

In [ ]:
# Confusion matrix with test set
results = confusion_matrix(predicted, test['overall'], labels=["Bad", "Neutral", "Good"])

ax = sns.heatmap(results, annot=True, cmap="Blues", fmt = '4d')
ax.set_title("Confusion Matrix")
ax.set_xlabel("Predicted Values")
ax.set_ylabel("True Values")

ax.xaxis.set_ticklabels(['Bad','Neutral', 'Good'])
ax.yaxis.set_ticklabels(['Bad','Neutral', 'Good'])

sns.set(rc={'figure.figsize':(8,6.5)})
# plt.figure(figsize=(100,100))
# plt.show()